In [ ]:
import pandas as pd
import xarray as xr
import os

In [ ]:
PATH = "../workflows/pypsa-eur-sec/"
OUTPUT = "../results/graphics-20221227/"

In [ ]:
# grid expansion, hydrogen network, costs, imports, shipping, onwind

SCENARIOS = {
    (
        1,
        1,
        0,
        0,
        0,
        0,
    ): f"{OUTPUT}/20221227-main/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/",
    (
        1,
        1,
        1,
        0,
        0,
        0,
    ): f"{OUTPUT}/20221227-costs/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/",
    (
        1,
        1,
        0,
        1,
        0,
        0,
    ): f"{OUTPUT}/20221227-import/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/",
    (
        1,
        1,
        0,
        0,
        1,
        0,
    ): f"{OUTPUT}/20221227-shipping/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/",
    (
        1,
        1,
        0,
        0,
        0,
        1,
    ): f"{OUTPUT}/20221227-main/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-onwind+p0_2050/",
    (
        1,
        0,
        0,
        0,
        0,
        0,
    ): f"{OUTPUT}/20221227-main/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/",
    (
        1,
        0,
        1,
        0,
        0,
        0,
    ): f"{OUTPUT}/20221227-costs/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/",
    (
        1,
        0,
        0,
        1,
        0,
        0,
    ): f"{OUTPUT}/20221227-import/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/",
    (
        1,
        0,
        0,
        0,
        1,
        0,
    ): f"{OUTPUT}/20221227-shipping/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/",
    (
        1,
        0,
        0,
        0,
        0,
        1,
    ): f"{OUTPUT}/20221227-main/elec_s_181_lvopt__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network-onwind+p0_2050/",
    (
        0,
        1,
        0,
        0,
        0,
        0,
    ): f"{OUTPUT}/20221227-main/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/",
    (
        0,
        1,
        1,
        0,
        0,
        0,
    ): f"{OUTPUT}/20221227-costs/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/",
    (
        0,
        1,
        0,
        1,
        0,
        0,
    ): f"{OUTPUT}/20221227-import/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/",
    (
        0,
        1,
        0,
        0,
        1,
        0,
    ): f"{OUTPUT}/20221227-shipping/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10_2050/",
    (
        0,
        1,
        0,
        0,
        0,
        1,
    ): f"{OUTPUT}/20221227-main/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-onwind+p0_2050/",
    (
        0,
        0,
        0,
        0,
        0,
        0,
    ): f"{OUTPUT}/20221227-main/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/",
    (
        0,
        0,
        1,
        0,
        0,
        0,
    ): f"{OUTPUT}/20221227-costs/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/",
    (
        0,
        0,
        0,
        1,
        0,
        0,
    ): f"{OUTPUT}/20221227-import/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/",
    (
        0,
        0,
        0,
        0,
        1,
        0,
    ): f"{OUTPUT}/20221227-shipping/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network_2050/",
    (
        0,
        0,
        0,
        0,
        0,
        1,
    ): f"{OUTPUT}/20221227-main/elec_s_181_lv1.0__Co2L0-3H-T-H-B-I-A-solar+p3-linemaxext10-noH2network-onwind+p0_2050/",
}

## Time Series

In [ ]:
datasets = [
    "total-electricity",
    "total-heat",
    "total-liquid-hydrocarbons",
    "H2",
    "gas",
    "co2 stored",
    "co2",
]

In [ ]:
data_vars = {}
for item in datasets:
    kwargs = dict(index_col=0, parse_dates=True)
    ts = {
        k: pd.read_csv(v + "balance-ts-" + item + ".csv", **kwargs)
        for k, v in SCENARIOS.items()
    }

    df = pd.concat(
        ts,
        names=[
            "power_grid",
            "hydrogen_grid",
            "optimistic_costs",
            "imports",
            "hydrogen_in_shipping",
            "no_onwind",
        ],
    )

    df.columns.name = "carrier"

    data_vars[item] = df.stack().to_xarray()

In [ ]:
ds = xr.Dataset(data_vars=data_vars)

In [ ]:
comp = dict(zlib=True, complevel=9)
encoding = {var: comp for var in ds.data_vars}
ds.to_netcdf("time-series.nc", encoding=encoding)

## Statistics

In [ ]:
kwargs = dict(index_col=0, header=[0, 1])
ts = {k: pd.read_csv(v + "report.csv", **kwargs) for k, v in SCENARIOS.items()}

df = pd.concat(
    ts,
    names=[
        "power_grid",
        "hydrogen_grid",
        "optimistic_costs",
        "imports",
        "hydrogen_in_shipping",
        "no_onwind",
    ],
).stack(level=1)

In [ ]:
df.index.names = df.index.names[:-2] + ["bus", "carrier"]

In [ ]:
ds = df.to_xarray()

comp = dict(zlib=True, complevel=9)
encoding = {var: comp for var in ds.data_vars}
ds.to_netcdf("report.nc", encoding=encoding)

## Electricity Network

In [ ]:
kwargs = dict(index_col=0, header=0)
ts = {
    k: pd.read_csv(v + "edges-electricity.csv", **kwargs) for k, v in SCENARIOS.items()
}

df = pd.concat(
    ts,
    names=[
        "power_grid",
        "hydrogen_grid",
        "optimistic_costs",
        "imports",
        "hydrogen_in_shipping",
        "no_onwind",
    ],
)
df.index.names = df.index.names[:-1] + ["line"]

In [ ]:
ds = df.to_xarray()

comp = dict(zlib=True, complevel=9)
encoding = {var: comp for var in ds.data_vars}
ds.to_netcdf("electricity-network.nc", encoding=encoding)

## Hydrogen Network

In [ ]:
kwargs = dict(index_col=0, header=0)
ts = {
    k: (
        pd.read_csv(v + "edges-hydrogen.csv", **kwargs)
        if os.path.exists(v + "edges-hydrogen.csv")
        else None
    )
    for k, v in SCENARIOS.items()
}

df = pd.concat(
    ts,
    names=[
        "power_grid",
        "hydrogen_grid",
        "optimistic_costs",
        "imports",
        "hydrogen_in_shipping",
        "no_onwind",
    ],
)
df.index.names = df.index.names[:-1] + ["line"]

In [ ]:
ds = df.to_xarray()

comp = dict(zlib=True, complevel=9)
encoding = {var: comp for var in ds.data_vars}
ds.to_netcdf("hydrogen-network.nc", encoding=encoding)

## Sankey

In [ ]:
ts = {
    k: pd.read_csv(v + "sankey.csv", index_col=0).reset_index(drop=True)
    for k, v in SCENARIOS.items()
}

df = pd.concat(
    ts,
    names=[
        "power_grid",
        "hydrogen_grid",
        "optimistic_costs",
        "imports",
        "hydrogen_in_shipping",
        "no_onwind",
    ],
)
df.index.names = df.index.names[:-1] + ["id"]

In [ ]:
ds = df.to_xarray()

comp = dict(zlib=True, complevel=9)
encoding = {var: comp for var in ds.data_vars}
ds.to_netcdf("energy-sankey.nc", encoding=encoding)

## Carbon Sankey

In [ ]:
ts = {
    k: pd.read_csv(v + "sankey-carbon.csv", index_col=0).reset_index(drop=True)
    for k, v in SCENARIOS.items()
}

df = pd.concat(
    ts,
    names=[
        "power_grid",
        "hydrogen_grid",
        "optimistic_costs",
        "imports",
        "hydrogen_in_shipping",
        "no_onwind",
    ],
)
df.index.names = df.index.names[:-1] + ["id"]

In [ ]:
ds = df.to_xarray()

comp = dict(zlib=True, complevel=9)
encoding = {var: comp for var in ds.data_vars}
ds.to_netcdf("carbon-sankey.nc", encoding=encoding)